In [ ]:
### 정적 페이지 크롤링하기 - 공공데이터

In [10]:
from bs4 import BeautifulSoup
import urllib.request as req

# API 요청 후, 응답결과를 xml 파일로 저장
url ="http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp"
filename = "forecast.xml"
req.urlretrieve(url, filename)

# 저장된 xml 파일을 읽어드려, 데이터 추출
info = {}
xml = open(filename, 'r', encoding="utf-8").read()
soup = BeautifulSoup(xml, 'xml')

# find_all("location") : <location> 태그들을 읽어옴
for location in soup.find_all("location"):
    city = location.find('city').string         # 도시
    min_t = location.find_all('tmn')            # 최저기온
    max_t = location.find_all('tmx')            # 최고기온
    date = location.find_all('tmEf')            # 날짜시간
    # 최저기온~최고기온 (10~25)
    # wether = [ a.string+"~"+b.string  for a, b in zip(min_t, max_t)]
    # print(f'{city} : {min_t} ~ {max_t}')
    # print(wether)
    wether = []
    # print( len(min_t) )
    # print( len(max_t) )
    # print( len(date) )
    for i in range( len(min_t) ):           # 13
        item = {}
        item['min_t'] = min_t[i].string
        item['max_t'] = max_t[i].string
        item['date'] = date[i].string
        wether.append(item)
    
    # print(city)
    # print(wether)
    
    if not (city in info):
        info[city] = []             # { '서울' : ['22~29'] }
    for data in wether:
        info[city].append(data)     # { '서울' : ['22~29', '22~29', '23~31', '23~31', ...]}
        
print(info)

{'서울': [{'min_t': '22', 'max_t': '29', 'date': '2023-06-30 00:00'}, {'min_t': '22', 'max_t': '29', 'date': '2023-06-30 12:00'}, {'min_t': '23', 'max_t': '31', 'date': '2023-07-01 00:00'}, {'min_t': '23', 'max_t': '31', 'date': '2023-07-01 12:00'}, {'min_t': '23', 'max_t': '30', 'date': '2023-07-02 00:00'}, {'min_t': '23', 'max_t': '30', 'date': '2023-07-02 12:00'}, {'min_t': '23', 'max_t': '30', 'date': '2023-07-03 00:00'}, {'min_t': '23', 'max_t': '30', 'date': '2023-07-03 12:00'}, {'min_t': '23', 'max_t': '29', 'date': '2023-07-04 00:00'}, {'min_t': '23', 'max_t': '29', 'date': '2023-07-04 12:00'}, {'min_t': '23', 'max_t': '29', 'date': '2023-07-05 00:00'}, {'min_t': '22', 'max_t': '28', 'date': '2023-07-06 00:00'}, {'min_t': '22', 'max_t': '29', 'date': '2023-07-07 00:00'}], '인천': [{'min_t': '22', 'max_t': '27', 'date': '2023-06-30 00:00'}, {'min_t': '22', 'max_t': '27', 'date': '2023-06-30 12:00'}, {'min_t': '22', 'max_t': '28', 'date': '2023-07-01 00:00'}, {'min_t': '22', 'max_t':

### 네이버 검색을 통한 서울, 부산, 제주도 날씨 정보 가져오기

In [32]:
from bs4 import BeautifulSoup
import urllib.request

# 서울 날씨
print('--------------- 서울 날씨')
html_text = urllib.request.urlopen('https://search.naver.com/search.naver?sm=top_hty&fbm=0&ie=utf8&query='+urllib.parse.quote_plus('서울날씨'))
soup = BeautifulSoup(html_text, 'html.parser')
temp = soup.select('.temperature_text > strong')
cast = soup.select('.weather.before_slash')
print( temp[0].contents[1] + '(' + cast[0].text +  ')' )

# 부산 날씨
print('--------------- 부산 날씨')
html_text = urllib.request.urlopen('https://search.naver.com/search.naver?sm=top_hty&fbm=0&ie=utf8&query='+urllib.parse.quote_plus('부산날씨'))
soup = BeautifulSoup(html_text, 'html.parser')
temp = soup.select('.temperature_text > strong')
cast = soup.select('.weather.before_slash')
print( temp[0].contents[1] + '(' + cast[0].text +  ')' )

# 제주도 날씨
print('--------------- 제주도 날씨')
html_text = urllib.request.urlopen('https://search.naver.com/search.naver?sm=top_hty&fbm=0&ie=utf8&query='+urllib.parse.quote_plus('제주도날씨'))
soup = BeautifulSoup(html_text, 'html.parser')
temp = soup.select('.temperature_text > strong')
cast = soup.select('.weather.before_slash')
print( temp[0].contents[1] + '(' + cast[0].text +  ')' )

--------------- 서울 날씨
24.8(흐림)
--------------- 부산 날씨
24.2(흐림)
--------------- 제주도 날씨
26.2(흐림)


### 버스 이용정보 조회하기 - 서울시 버스 정보

In [58]:
from bs4 import BeautifulSoup
import urllib.request as req

# 버스번호로 busRouteId 가져오기
busNum = '6628'
key = '%2BjzsSyNtwmcqxUsGnflvs3rW2oceFvhHR8AFkM3ao%2Fw50hwHXgGyPVutXw04uAXvrkoWgkoScvvhlH7jgD4%2FRQ%3D%3D'
url1 = 'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?serviceKey='+key+'&strSrch='+busNum
res = req.urlopen(url1)

result = res.read().decode('utf-8')

soup = BeautifulSoup( result, 'xml')

busRouteId = None
for itemList in soup.find_all('itemList'):
    busRouteId = itemList.find('busRouteId').string
    busRouteNm = itemList.find('busRouteNm').string
    if busRouteNm == busNum :
        break
    
print(f' busRouteId : {busRouteId}')
print(f' busRouteNm : {busRouteNm}')



# 노선별 경유 정류소 조회 서비스

url2 = 'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?serviceKey='+key+'&busRouteId='+busRouteId
res = req.urlopen(url2)
result = res.read()
print(result)
soup = BeautifulSoup( result, 'xml' )

# print( soup.prettify() )

busPos = []
for itemList in soup.find_all('itemList'):
    stationNm = itemList.find('stationNm').string
    gpsY = itemList.find('gpsY').string
    gpsX = itemList.find('gpsX').string
    beginTm = itemList.find('beginTm').string
    lastTm = itemList.find('lastTm').string
    
    busPos.append( (stationNm, gpsY, gpsX, beginTm, lastTm) )
    
print( busNum + '번 버스 정류소 위치' )
for stationNm, lat, lng, beginTm, lastTm in busPos:
    print( stationNm, ':' , lat +','+ lng + ' (' + beginTm + '~' + lastTm + ')' )      # 위도, 경도

   

 busRouteId : 100100305
 busRouteNm : 6628
b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><ServiceResult><comMsgHeader/><msgHeader><headerCd>0</headerCd><headerMsg>\xec\xa0\x95\xec\x83\x81\xec\xa0\x81\xec\x9c\xbc\xeb\xa1\x9c \xec\xb2\x98\xeb\xa6\xac\xeb\x90\x98\xec\x97\x88\xec\x8a\xb5\xeb\x8b\x88\xeb\x8b\xa4.</headerMsg><itemCount>0</itemCount></msgHeader><msgBody><itemList><arsId>16408</arsId><beginTm>04:20</beginTm><busRouteAbrv>6628</busRouteAbrv><busRouteId>100100305</busRouteId><busRouteNm>6628</busRouteNm><direction>\xec\x97\xac\xec\x9d\x98\xeb\x8f\x84</direction><gpsX>126.8213321956</gpsX><gpsY>37.5452091516</gpsY><lastTm>23:30</lastTm><posX>184212.7708663494</posX><posY>449542.1810359913</posY><routeType>4</routeType><sectSpd>0</sectSpd><section>0</section><seq>1</seq><station>115000296</station><stationNm>\xec\x99\xb8\xeb\xb0\x9c\xec\x82\xb0\xeb\x8f\x99\xea\xb8\xb0\xec\xa0\x90</stationNm><stationNo>16408</stationNo><transYn>N</transYn><fullSectDist>0</fullSectDist><t

### 네이버 블로그 검색

In [ ]:
import requests
import urllib
import re

# 네이버 애플리케이션 등록 - client_id, client_secret
client_id = '4kU0V67743chqvBsEt3R'
client_secret = 'HNv3hzrPqP'

query = '영등포 맛집'
query = urllib.parse.quote_plus(query)
display = str(20)

url = 'https://openapi.naver.com/v1/search/blog.json?query=' + query + '&display=' + display
headers = {'X-Naver-Client-Id' : client_id, 'X-Naver-Client-Secret' : client_secret}

res = requests.get( url, headers=headers ) 
status = res.status_code

if( status == 200 ):
    data_result = res.json()
    # print(data_result)
    for count, data in enumerate(data_result['items'], 1):
        title = re.sub("</?b>", "", data['title'] )                 # 제목에서 <b> </b> 제거
        description = re.sub("</?b>", "", data['description'] )     # 내용에서 <b> </b> 제거
        print(str(count) + ' : ' + title)
        print('------------------------')
        print( description )
        print( '블로그명 : ' + data['bloggername'])
        print( 'url : ' + data['link'] )

else:
    print('요청 실패 : ' + status)

### 네이버 뉴스 기사 검색

In [70]:
import urllib
import json

# 네이버 애플리케이션 등록 - client_id, client_secret
client_id = '4kU0V67743chqvBsEt3R'
client_secret = 'HNv3hzrPqP'

query = '올해의 '
query = urllib.parse.quote_plus(query)
display = str(20)

url = 'https://openapi.naver.com/v1/search/news.json?query=' + query + '&display=' + display
# headers = {'X-Naver-Client-Id' : client_id, 'X-Naver-Client-Secret' : client_secret}

request = urllib.request.Request(url)
request.add_header('X-Naver-Client-Id', client_id)
request.add_header('X-Naver-Client-Secret', client_secret)

res = urllib.request.urlopen(request)
ststus = res.getcode()

if( status == 200 ):
    result = res.read()
    result = json.loads(result)
    
    count = 1
    for data in result['items']:
        print( str(count) + ' : ' + data['title'] )
        print('=====================================')
        print(': ' + data['description'])
        count += 1
    # print(result)

else:
    print('요청 실패 : ' + status)

1 : 경기도, 영유아발달지원서비스 3년간 7천300명에 제공
: 사업 첫해인 2021년 3천759명(신청자 중 475명 전문기관 연계), 2022년 2천595명(신청자 중 871명 전문기관 연계), <b>올해</b> 5월 기준 1천27명이다. <b>올해</b> 신청자 1천27명 중 언어·발달 지연 문제는 557명, 정서 관련 문제는 470명으로 이... 
2 : 창립 10주년 JB금융그룹…작지만 젊고 강한 &apos;강소금융그룹&apos; 도약
: ▲ 김기홍 회장출범 첫 해 16조1861억 원 수준이던 그룹 총자산은 <b>올해</b> 1분기 60조 원을 돌파하며 3.7배... 39%를 기록한 데 이어 <b>올해</b> 3월에는 12.32%를 달성했다. 주요 경영 지표인 자기자본이익률(ROE)은 지난해 13.88... 
3 : 총수 보수·배당금 인상 YES, 라면값 인하 NO
: <b>올해</b> 1분기 라면 소비자물가 상승률이 전체 가구 소득 증가율 대비 3배가 넘는 상황이다. 가격인상 효과는 달콤했다. 농심과 오뚜기는 지난해 매출이 3조원을 돌파하면서 역대 최대의 실적을 올렸고 <b>올해</b>도 상황은... 
4 : &quot;CJ ONE 포인트, HD현대오일뱅크 주유소서 사용하세요&quot;
: 특히 <b>올해</b> 들어 ▲편의점 &apos;CU&apos; ▲음원 사이트 &apos;벅스&apos; ▲여행 서비스 &apos;트립닷컴&apos;까지 일상생활에서 쉽게 접할 수 있어 고객 선호도가 높은 생활 편의, 문화, 여행 분야의 제휴처를 확대했다. CJ ONE 포인트는 30여 곳의... 
5 : 한은·KDI “하반기 정부 소비 2%대까지 떨어질 것”
: 연합뉴스 주요 경제전망 기관들이 <b>올해</b> 하반기 경제 상황에 대해 부정적으로 전망했다. 정부가 하반기... 해당 자료에 따르면 한은과 KDI는 <b>올해</b> 하반기 정부 소비 성장률가 2%대에 머물 것으로 전망했다. 한은과 KDI는... 
6 : 논산시, 농업기계 임대료 감면 혜택 연장
: 